A primer on using [RDKit](https://www.rdkit.org/ )
---

This is a popular package for cheminformatics calculations and manipulations. The [documentation](https://rdkit.org/docs/index.html) contains many examples, recipes, and links to other resources. A full history is given [here](https://rdkit.org/docs/Overview.html#open-source-toolkit-for-cheminformatics). Due to its popularity, many other open source tools used RDKit and related/compatible data structures.

The [RDKit book](https://rdkit.org/docs/RDKit_Book.html) explains conventions, approaches, and features.

These notes will focus on using rdkit in python and focus on only portions of the toolkit. The [cookbook](https://rdkit.org/docs/Cookbook.html) has a lot of recipes, many of which are used below. RDKit provides attribution for these original examples, which may be modified below.

In [ ]:
!pip install rdkit

In [ ]:
import rdkit 
rdkit.__version__

# The Basics

> "Chem vs AllChem: The majority of “basic” chemical functionality (e.g. reading/writing molecules, substructure searching, molecular cleanup, etc.) is in the rdkit.Chem module. More advanced, or less frequently used, functionality is in rdkit.Chem.AllChem. The distinction has been made to speed startup and lower import times; there’s no sense in loading the 2D->3D library and force field implementation if one is only interested in reading and writing a couple of molecules. If you find the Chem/AllChem thing annoying or confusing, you can use python’s “import … as …” syntax to remove the irritation."

~~~python
from rdkit.Chem import AllChem as Chem
~~~

Otherwise you can just import the Chem module.  I personally, do not notice a substantial difference.

There are 2 basic languages used to describe molecules: [SMILES](https://www.daylight.com/dayhtml/doc/theory/theory.smiles.html) and [SMARTS](https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html)


## SMILES

* The original SMILES specification was developed by a company called Daylight; their description is [here](https://www.daylight.com/dayhtml/doc/theory/theory.smiles.html).

* The [opensmiles](http://opensmiles.org/) specification is described [here](http://opensmiles.org/opensmiles.html)

* Wikipedia's description is [here](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system).

> "In terms of a graph-based computational procedure, SMILES is a string obtained by printing the symbol nodes encountered in a depth-first tree traversal of a chemical graph. The chemical graph is first trimmed to remove hydrogen atoms and cycles are broken to turn it into a spanning tree. Where cycles have been broken, numeric suffix labels are included to indicate the connected nodes. Parentheses are used to indicate points of branching on the tree.
>
> The resultant SMILES form depends on the choices:
>
> * of the bonds chosen to break cycles,
> * of the starting atom used for the depth-first traversal, and
> * of the order in which branches are listed when encountered.

There is a "canonical" form which standardizes this and is unique.  There are a number of different algorithms for canonicalization, and they may not produce the same results as each other.

In [ ]:
from rdkit import Chem
mol = Chem.MolFromSmiles('CC(N)C1CC1')
Chem.MolToRandomSmilesVect(mol, 5, randomSeed=42)

In [ ]:
smis = []
for i in range(5):
    smis.append(Chem.MolToSmiles(mol, doRandom=True, canonical=False)) # Do not use the canonical form
smis

In [ ]:
smi = "CN1C(NC2=NC=CC=C2)=CC=C1"
mol = Chem.MolFromSmiles(smi)
print(Chem.MolToSmiles(mol))

In [ ]:
Chem.Kekulize(mol) # Kekule SMILES
print(Chem.MolToSmiles(mol, kekuleSmiles=True))

In [ ]:
?Chem.Kekulize

In [ ]:
# You can detect chemistry problems automatically
from rdkit import Chem
m = Chem.MolFromSmiles('CN(C)(C)C', sanitize=False)
problems = Chem.DetectChemistryProblems(m)

In [ ]:
problems[0].GetType()

In [ ]:
problems[0].GetAtomIdx()

In [ ]:
problems[0].Message()

In [ ]:
m = Chem.MolFromSmiles('CN(C)(C)C', sanitize=True)

In [ ]:
type(m) # Santization does not return an invalid molecule

## SMARTS

# Visualization
Let's look at some existing molecules and how to manipulate them.

## Drawing a molecule

### Option 1: Map index to atom location directly

In [ ]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
IPythonConsole.ipython_useSVG=True  #< set this to False if you want PNGs instead of SVGs

In [ ]:
def mol_with_atom_index(mol):
    for atom in mol.GetAtoms():
        atom.SetAtomMapNum(atom.GetIdx())
    return mol

In [ ]:
# Test in a kinase inhibitor
mol = Chem.MolFromSmiles("C1CC2=C3C(=CC=C2)C(=CN3C1)[C@H]4[C@@H](C(=O)NC4=O)C5=CNC6=CC=CC=C65")
# Default
mol

In [ ]:
# With atom index
mol_with_atom_index(mol)

### Option 2: Using Jupyter display settings directly (better)

In [ ]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
IPythonConsole.drawOptions.addAtomIndices = True
IPythonConsole.molSize = 300,300

In [ ]:
mol = Chem.MolFromSmiles("C1CC2=C3C(=CC=C2)C(=CN3C1)[C@H]4[C@@H](C(=O)NC4=O)C5=CNC6=CC=CC=C65")
mol

In [ ]:
IPythonConsole.drawOptions.addAtomIndices = False

## Display calculated features and annotations

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.molSize = 250,250

In [ ]:
m = Chem.MolFromSmiles('c1ncncc1C(=O)[O-]')
AllChem.ComputeGasteigerCharges(m)
m

In [ ]:
# The 'atomNote' property lets you add arbitrary annotations
# There is an analogous 'bondNote' too (e.g., mol.GetBondWithIdx(0).SetProp('bondNote', 'bar'))
m2 = Chem.Mol(m)
for at in m2.GetAtoms():
    lbl = '%.2f'%(at.GetDoubleProp("_GasteigerCharge"))
    at.SetProp('atomNote',lbl)
m2

In [ ]:
# You can combine this with other annotations - by default they are separated by a comma
IPythonConsole.drawOptions.addAtomIndices = True
m2

In [ ]:
# You can also add sub/superscripts to your annotations
smi = 'c1nc(*)ccc1* |$;;;R1;;;;R2$|'
mol = Chem.MolFromSmiles(smi)
mol.GetAtomWithIdx(3).SetProp("_displayLabel","CO<sub>2</sub>H")
mol.GetAtomWithIdx(3).SetProp("_displayLabelW","HO<sub>2</sub>C")
mol.GetAtomWithIdx(7).SetProp("_displayLabel","CO<sub>2</sub><sup>-</sup>")
mol.GetAtomWithIdx(7).SetProp("_displayLabelW","<sup>-</sup>OOC")
mol

In [ ]:
IPythonConsole.drawOptions.addAtomIndices = False

## Stereochemistry

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.drawOptions.addAtomIndices = False
IPythonConsole.drawOptions.addStereoAnnotation = True

In [ ]:
# Default Representation uses legacy FindMolChiralCenters() code
m1 = Chem.MolFromSmiles('C1CC1[C@H](F)C1CCC1')
m2 = Chem.MolFromSmiles('F[C@H]1CC[C@H](O)CC1')
Draw.MolsToGridImage((m1,m2), subImgSize=(250,250))

In [ ]:
# new stereochemistry code with more accurate CIP labels, 2020.09 release
from rdkit.Chem import rdCIPLabeler
rdCIPLabeler.AssignCIPLabels(m1)
rdCIPLabeler.AssignCIPLabels(m2)
Draw.MolsToGridImage((m1,m2), subImgSize=(250,250))

## Highlighting substructures

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.drawOptions.addAtomIndices = True

In [ ]:
m = Chem.MolFromSmiles('c1cc(C(=O)O)c(OC(=O)C)cc1')
substructure = Chem.MolFromSmarts('C(=O)O')

In [ ]:
Draw.MolsToGridImage((m, substructure))

In [ ]:
subs = m.GetSubstructMatches(substructure)
# ?m.GetSubstructMatches
# Returns tuples of the indices of the molecule's atoms that match a substructure query.

In [ ]:
subs

In [ ]:
m # The structure is colored in-place

In [ ]:
Chem.MolFragmentToSmiles(m, subs[0])

In [ ]:
Chem.MolFragmentToSmiles(m, subs[1])

In [ ]:
# You can also manually set the atoms that should be highlighted:
m.__sssAtoms = [0,1,2,6,11,12]
m

In [ ]:
IPythonConsole.drawOptions.addAtomIndices = False

## Highlighting differences between molecules

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdFMCS
from rdkit.Chem.Draw import rdDepictor
rdDepictor.SetPreferCoordGen(True)
IPythonConsole.drawOptions.minFontSize=20

In [ ]:
mol1 = Chem.MolFromSmiles('FC1=CC=C2C(=C1)C=NN2')
mol2 = Chem.MolFromSmiles('CCC1=C2NN=CC2=CC(Cl)=C1')

In [ ]:
Draw.MolsToGridImage([mol1, mol2])

In [ ]:
# This function will find the maximum common substructure
# See documentation for more details: https://rdkit.org/docs/GettingStartedInPython.html#maximum-common-substructure

def view_difference(mol1, mol2):
    mcs = rdFMCS.FindMCS([mol1,mol2])
    mcs_mol = Chem.MolFromSmarts(mcs.smartsString)

    match1 = mol1.GetSubstructMatch(mcs_mol)
    target_atm1 = []
    for atom in mol1.GetAtoms():
        if atom.GetIdx() not in match1:
            target_atm1.append(atom.GetIdx())

    match2 = mol2.GetSubstructMatch(mcs_mol)
    target_atm2 = []
    for atom in mol2.GetAtoms():
        if atom.GetIdx() not in match2:
            target_atm2.append(atom.GetIdx())

    return Draw.MolsToGridImage([mol1, mol2], highlightAtomLists=[target_atm1, target_atm2])

In [ ]:
view_difference(mol1,mol2)

In [ ]:
rdDepictor.SetPreferCoordGen(False)

## Elaborate (multi)coloring

In [ ]:
# It is possible to color atoms multiple ways if they fall into multiple sets

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.Draw import IPythonConsole
from IPython.display import SVG

smi = 'c1cc(F)ccc1Cl'
mol = Chem.MolFromSmiles(smi)
patt = Chem.MolFromSmarts('ClccccF')
hit_ats = list(mol.GetSubstructMatch(patt))

hit_bonds = []
for bond in patt.GetBonds():
  aid1 = hit_ats[bond.GetBeginAtomIdx()]
  aid2 = hit_ats[bond.GetEndAtomIdx()]
  hit_bonds.append(mol.GetBondBetweenAtoms(aid1,aid2).GetIdx())

In [ ]:
d = rdMolDraw2D.MolDraw2DSVG(250, 250) # or MolDraw2DCairo to get PNGs
rdMolDraw2D.PrepareAndDrawMolecule(d, mol, highlightAtoms=hit_ats, highlightBonds=hit_bonds)
d.FinishDrawing()
SVG(d.GetDrawingText())

In [ ]:
def colors(index):
  # A simple function that returns a color based on an index
  if index%4 == 0:
    return (0.8,0.0,0.8)
  elif index%4 == 1:
    return (0.8,0.8,0)
  elif index%4 == 2: 
    return (0,0.8,0.8)
  else: 
    return (0,0,0.8)
  
atom_cols = {}
for i, at in enumerate(hit_ats):
  atom_cols[at] = colors(i)

bond_cols = {}
for i, bd in enumerate(hit_bonds):
  bond_cols[bd] = colors(3 - i%4)

In [ ]:
d = rdMolDraw2D.MolDraw2DSVG(250, 250) # or MolDraw2DCairo to get PNGs
rdMolDraw2D.PrepareAndDrawMolecule(d, mol, highlightAtoms=hit_ats,
                                   highlightAtomColors=atom_cols,
                                   highlightBonds=hit_bonds,
                                   highlightBondColors=bond_cols)
d.FinishDrawing()
SVG(d.GetDrawingText())

In [ ]:
# See https://github.com/rdkit/rdkit/blob/master/Docs/Book/data/test_multi_colours.py for an even more elaborate example

## Functional group abbreviations

In [ ]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Chem import rdAbbreviations

In [ ]:
m = Chem.MolFromSmiles('COc1ccc(C(=O)[O-])cc1')
m

In [ ]:
abbrevs = rdAbbreviations.GetDefaultAbbreviations()
nm = rdAbbreviations.CondenseMolAbbreviations(m,abbrevs)
nm

In [ ]:
# abbreviations that cover more than 40% of the molecule won't be applied by default
m = Chem.MolFromSmiles('c1c[nH]cc1C(F)(F)F')
nm1 = rdAbbreviations.CondenseMolAbbreviations(m,abbrevs)
nm2 = rdAbbreviations.CondenseMolAbbreviations(m,abbrevs,maxCoverage=0.8)
Draw.MolsToGridImage((m,nm1,nm2),legends=('','default','maxCoverage=0.8'))

In [ ]:
# See available abbreviations
abbrevs = rdAbbreviations.GetDefaultAbbreviations()
for a in abbrevs:
    print(a.label)

## Representing large molecules

> "Some molecules like macrocycles are not represented well using the default RDKit drawing code. As a result, it may be preferable to use the CoordGen integration."

You can set this preference globally like:
~~~python
from rdkit.Chem.Draw import rdDepictor
rdDepictor.SetPreferCoordGen(True)
~~~

In [ ]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.molSize = 350,300
from rdkit.Chem import Draw

In [ ]:
# default drawing
mol = Chem.MolFromSmiles("C/C=C/CC(C)C(O)C1C(=O)NC(CC)C(=O)N(C)CC(=O)N(C)C(CC(C)C)C(=O)NC(C(C)C)C(=O)N(C)C(CC(C)C)C(=O)NC(C)C(=O)NC(C)C(=O)N(C)C(CC(C)C)C(=O)N(C)C(CC(C)C)C(=O)N(C)C(C(C)C)C(=O)N1C")
mol

In [ ]:
from rdkit.Chem import rdCoordGen
rdCoordGen.AddCoords(mol)
mol

## Integrating with matplotlib

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole

In [ ]:
x = np.arange(0, 180, 1)
y = np.sin(x)

In [ ]:
mol = Chem.MolFromSmiles('C1CNCCC1C(=O)C')
im = Chem.Draw.MolToImage(mol) # Convert to image

In [ ]:
type(im)

In [ ]:
fig = plt.figure(figsize=(10,5))
plt.plot(x, y)
plt.ylim(-1, 5)
ax = plt.axes([0.6, 0.47, 0.38, 0.38], frameon=True)
ax.imshow(im) # Use imshow to display
ax.axis('off')

# Identifying features
Let's see to look at what features a molecule has, including properties of the atoms that make up the molecule.

## Counting ring systems

In [ ]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole

In [ ]:
def GetRingSystems(mol, includeSpiro=False):
    ri = mol.GetRingInfo()
    systems = []
    for ring in ri.AtomRings():
        ringAts = set(ring)
        nSystems = []
        for system in systems:
            nInCommon = len(ringAts.intersection(system))
            if nInCommon and (includeSpiro or nInCommon>1):
                ringAts = ringAts.union(system)
            else:
                nSystems.append(system)
        nSystems.append(ringAts)
        systems = nSystems
    return systems

In [ ]:
mol = Chem.MolFromSmiles('CN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3')
print(GetRingSystems(mol))

In [ ]:
IPythonConsole.drawOptions.addAtomIndices = True
mol

In [ ]:
IPythonConsole.drawOptions.addAtomIndices = False

## Identify aromatic rings

https://rdkit.org/docs/RDKit_Book.html#aromaticity

In [ ]:
from rdkit import Chem
IPythonConsole.drawOptions.addAtomIndices = True
IPythonConsole.drawOptions.addBondIndices = True
m = Chem.MolFromSmiles('c1cccc2c1CCCC2')
m

In [ ]:
ri = m.GetRingInfo()
# You can interrogate the RingInfo object to tell you the atoms that make up each ring:
print(ri.AtomRings())

In [ ]:
# or the bonds that make up each ring:
print(ri.BondRings())

In [ ]:
# To detect aromatic rings, loop over the bonds in each ring and
# flag the ring as aromatic if all bonds are aromatic:
def isRingAromatic(mol, bondRing):
        for id in bondRing:
            if not mol.GetBondWithIdx(id).GetIsAromatic():
                return False
        return True

In [ ]:
print(isRingAromatic(m, ri.BondRings()[0]))

In [ ]:
print(isRingAromatic(m, ri.BondRings()[1]))

## Identify atom properties

### Direct method

In [ ]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
IPythonConsole.drawOptions.addAtomIndices = True

In [ ]:
mol = Chem.MolFromSmiles("c1ccccc1C=CCC")
aromatic_carbon = Chem.MolFromSmarts("c")
print(mol.GetSubstructMatches(aromatic_carbon))

In [ ]:
mol

In [ ]:
# The RDKit includes a SMARTS extension that allows hybridization queries,
# here we query for SP2 aliphatic carbons:
olefinic_carbon = Chem.MolFromSmarts("[C^2]")
print(mol.GetSubstructMatches(olefinic_carbon))

In [ ]:
IPythonConsole.drawOptions.addAtomIndices = False

### Using rdqueries

 See [rdqueries](https://rdkit.org/docs/source/rdkit.Chem.rdqueries.html?highlight=rdqueries) module for more examples on how to use these query tools.

In [ ]:
from rdkit.Chem import rdqueries

In [ ]:
mol = Chem.MolFromSmiles("c1ccccc1C=CCC")
q = rdqueries.IsAromaticQueryAtom()
print([x.GetIdx() for x in mol.GetAtomsMatchingQuery(q)])

In [ ]:
q = rdqueries.HybridizationEqualsQueryAtom(Chem.HybridizationType.SP2)
print([x.GetIdx() for x in mol.GetAtomsMatchingQuery(q)])

In [ ]:
qcombined = rdqueries.IsAliphaticQueryAtom()
qcombined.ExpandQuery(q)
print([x.GetIdx() for x in mol.GetAtomsMatchingQuery(qcombined)])

## Substructure matching

In [ ]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.drawOptions.addAtomIndices = True

sucrose = "C([C@@H]1[C@H]([C@@H]([C@H]([C@H](O1)O[C@]2([C@H]([C@@H]([C@H](O2)CO)O)O)CO)O)O)O)O"
sucrose_mol = Chem.MolFromSmiles(sucrose)
primary_alcohol = Chem.MolFromSmarts("[CH2][OH1]")

sucrose_mol

In [ ]:
primary_alcohol

In [ ]:
print(sucrose_mol.GetSubstructMatches(primary_alcohol))

In [ ]:
secondary_alcohol = Chem.MolFromSmarts("[CH1][OH1]")
print(sucrose_mol.GetSubstructMatches(secondary_alcohol))

In [ ]:
# Macrocycles
erythromycin = Chem.MolFromSmiles("CC[C@@H]1[C@@]([C@@H]([C@H](C(=O)[C@@H](C[C@@]([C@@H]([C@H]([C@@H]([C@H](C(=O)O1)C)O[C@H]2C[C@@]([C@H]([C@@H](O2)C)O)(C)OC)C)O[C@H]3[C@@H]([C@H](C[C@H](O3)C)N(C)C)O)(C)O)C)C)O)(C)O")
erythromycin

In [ ]:
# Define SMARTS pattern with ring size > 12
# This is an RDKit SMARTS extension
macro = Chem.MolFromSmarts("[r{12-}]")
print(erythromycin.GetSubstructMatches(macro))

In [ ]:
erythromycin

In [ ]:
# Match only within a fragment

# Define a function where matches are contained in a single fragment
def fragsearch(m,p):
    matches = [set(x) for x in m.GetSubstructMatches(p)]
    frags = [set(y) for y in Chem.GetMolFrags(m)] # had to add this line for code to work
    for frag in frags:
        for match in matches:
            if match.issubset(frag):
                return match
    return False

In [ ]:
p = Chem.MolFromSmarts('O.N')
m1 = Chem.MolFromSmiles('OCCCN.CCC') # Set of 2 molecules
m2 = Chem.MolFromSmiles('OCCC.CCCN') # Set of another 2 molecules

Draw.MolsToGridImage((p, m1, m2))

In [ ]:
print(m1.HasSubstructMatch(p), fragsearch(m1, p))

In [ ]:
m1.GetSubstructMatch(p)

In [ ]:
print(m2.HasSubstructMatch(p), fragsearch(m2, p)) # Requies the are both part of same fragment

In [ ]:
m2.GetSubstructMatch(p) # Does not search within same fragment

In [ ]:
IPythonConsole.drawOptions.addAtomIndices = False

# Manipulating molecules
Now, let's look at how to make your own molecules and bonds from scratch.

## Creating a molecule

In [ ]:
# You can create a molecule is lots of ways, but if it fails it will always return None
good_mol = Chem.MolFromSmiles('Cc1ccccc1') # This always returns canonical version by default so it is reproducible
bad_mol = Chem.MolFromSmiles('CO(C)C')
# mol = Chem.MolFromMolFile('data/chiral.mol')

In [ ]:
good_mol

In [ ]:
type(bad_mol)

Akin to Datasets in Keras, you can stream molecules from files either in raw ASCII with [SmilesMolSupplier](https://www.rdkit.org/docs/source/rdkit.Chem.rdmolfiles.html#rdkit.Chem.rdmolfiles.SmilesMolSupplier) or from a compressed binary file with [ForwardSDMolSupplier](https://www.rdkit.org/docs/source/rdkit.Chem.rdmolfiles.html#rdkit.Chem.rdmolfiles.ForwardSDMolSupplier)

Best practice is to check if the molecule is "good" when read.

~~~python
with Chem.SDMolSupplier('data/5ht3ligs.sdf') as suppl:
  for mol in suppl:
    if mol is None: continue
    print(mol.GetNumAtoms())
~~~

There are MultithreadedSDMolSupplier() versions whose access order becomes random, but at the time of writing this was still experimental.

In [ ]:
?Chem.MolToSmiles # To see documentation on converting to a SMILES string

In [ ]:
# You can create a molecule directly using the RWMol module


## Looping over parts of a molecule

In [ ]:
m = Chem.MolFromSmiles('C1OC1')
for atom in m.GetAtoms():
  print(atom.GetAtomicNum())

In [ ]:
m

In [ ]:
print(
    m.GetAtomWithIdx(0).GetSymbol(),
    m.GetAtomWithIdx(0).GetExplicitValence(),
    m.GetBondWithIdx(0).GetBeginAtomIdx(),
    m.GetBondWithIdx(0).GetEndAtomIdx(),
    m.GetBondBetweenAtoms(0,1).GetBondType()
)

In [ ]:
# Atoms track their neighbors
[a.GetSymbol() for a in m.GetAtomWithIdx(0).GetNeighbors()]

## Re-order atoms

In [ ]:
from rdkit import Chem
from rdkit.Chem.Draw import MolsToGridImage

In [ ]:
m = Chem.MolFromSmiles("c1([C@H](C)CC)cccc2ccccc12")
m1 = Chem.MolFromSmiles("c12ccccc1c(ccc2)[C@H](C)CC")
print(Chem.MolToSmiles(m) == Chem.MolToSmiles(m1))

In [ ]:
# check if current canonical atom ordering matches
m_neworder = tuple(zip(*sorted([(j, i) for i, j in enumerate(Chem.CanonicalRankAtoms(m))])))[1]
m1_neworder = tuple(zip(*sorted([(j, i) for i, j in enumerate(Chem.CanonicalRankAtoms(m1))])))[1]
print(m_neworder == m1_neworder)

In [ ]:
# add atom numbers in images
def addAtomIndices(mol):
    for i, a in enumerate(mol.GetAtoms()):
        a.SetAtomMapNum(i)

In [ ]:
addAtomIndices(m)
addAtomIndices(m1)
MolsToGridImage((m, m1))

In [ ]:
# renumber atoms with same canonical ordering
m_renum = Chem.RenumberAtoms(m, m_neworder)
m1_renum = Chem.RenumberAtoms(m1, m1_neworder)
addAtomIndices(m_renum)
addAtomIndices(m1_renum)
MolsToGridImage((m_renum, m1_renum))

## Fragmenting a molecule

### Fragmentation tools

In [ ]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole, MolsToGridImage
IPythonConsole.drawOptions.addBondIndices = True
IPythonConsole.drawOptions.addAtomIndices = True

# I have put explicit bonds in the SMILES definition to facilitate comprehension:
mol = Chem.MolFromSmiles("O-C-C-C-C-N")
mol1 = Chem.Mol(mol)
mol2 = Chem.Mol(mol)
mol1

In [ ]:
# Chem.FragmentOnBonds() will fragment all specified bond indices at once, and return a single molecule
# with all specified cuts applied. By default, addDummies=True, so empty valences are filled with dummy atoms:
mol1_f = Chem.FragmentOnBonds(mol1, (0, 2, 4))
mol1_f

In [ ]:
# This molecule can be split into individual fragments using Chem.GetMolFrags():
MolsToGridImage(Chem.GetMolFrags(mol1_f, asMols=True))

In [ ]:
# Chem.FragmentOnSomeBonds() will fragment according to all permutations of numToBreak bonds at a time
# (numToBreak defaults to 1), and return tuple of molecules with numToBreak cuts applied. By default,
# addDummies=True, so empty valences are filled with dummy atoms:
mol2_f_tuple = Chem.FragmentOnSomeBonds(mol2, (0, 2, 4), numToBreak=1)

In [ ]:
Draw.MolsToGridImage(Chem.GetMolFrags(mol2_f_tuple[0], asMols=True), subImgSize=(250,250))

In [ ]:
Draw.MolsToGridImage(Chem.GetMolFrags(mol2_f_tuple[1], asMols=True), subImgSize=(250,250))

In [ ]:
Draw.MolsToGridImage(Chem.GetMolFrags(mol2_f_tuple[2], asMols=True), subImgSize=(250,250))

### Manually remove bonds

In [ ]:
# Finally, you can manually cut bonds using Chem.RWMol.RemoveBonds:
with Chem.RWMol(mol) as rwmol:
  for b_idx in [0, 2, 4]:
    b = rwmol.GetBondWithIdx(b_idx)
    rwmol.RemoveBond(b.GetBeginAtomIdx(), b.GetEndAtomIdx())
# And then call Chem.GetMolFrags() to get sanitized fragments where empty valences were filled with implicit hydrogens:
MolsToGridImage(Chem.GetMolFrags(rwmol, asMols=True))

In [ ]:
IPythonConsole.drawOptions.addBondIndices = False
IPythonConsole.drawOptions.addAtomIndices = False

### Murcko decomposition

In [ ]:
# RDKit provides standard Murcko-type decomposition of molecules into scaffolds:

import requests
r = requests.get('https://raw.githubusercontent.com/rdkit/rdkit/master/Docs/Book/data/cdk2.sdf')
f = open('cdk2.sdf', 'w')
f.write(r.text)
f.close()

In [ ]:
from rdkit.Chem.Scaffolds import MurckoScaffold
with Chem.SDMolSupplier('cdk2.sdf') as cdk2mols:
  m1 = cdk2mols[0]
core = MurckoScaffold.GetScaffoldForMol(m1)
Chem.MolToSmiles(core)

In [ ]:
core

In [ ]:
# Or into a generic framework
fw = MurckoScaffold.MakeScaffoldGeneric(core)
Chem.MolToSmiles(fw)

In [ ]:
fw

## Enumerating products / functionalized results

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem

from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.drawOptions.addAtomIndices = True

In [ ]:
# Replace sidechains on a core and enumerate the combinations
core = '*c1c(C)cccc1(O)'
core_mol = Chem.MolFromSmiles(core)
chain = 'CN*'
chain_mol = Chem.MolFromSmiles(chain)

In [ ]:
Draw.MolsToGridImage((core_mol, chain_mol), subImgSize=(250,250))

In [ ]:
rxn = AllChem.ReactionFromSmarts('[c:1][#0].[#0][*:2]>>[c:1]-[*:2]')
reacts = (core_mol, chain_mol)
products = rxn.RunReactants(reacts) # tuple
print(len(products))

In [ ]:
print(Chem.MolToSmiles(products[0][0]))

In [ ]:
products[0][0]

In [ ]:
# The above reaction-based approach is flexible, however if you can generate your
# sidechains in such a way that the atom you want to attach to the core
# is the first one (atom zero), there's a somewhat easier way to do this
# kind of simple replacement:

products = Chem.ReplaceSubstructs(core_mol, Chem.MolFromSmarts('[#0]'), chain_mol) # tuple
print(Chem.MolToSmiles(products[0]))

# Can also do things with Chem.DeleteSubstructs

In [ ]:
products[0]

In [ ]:
# Here is an example in a loop for an imidazolium core with alkyl chains

core_mol = Chem.MolFromSmiles('*[n+]1cc[nH]c1')
chains = ['C','CC','CCC','CCCC','CCCCC','CCCCCC']
chainMols = [Chem.MolFromSmiles(chain) for chain in chains]

product_smi = []
for chainMol in chainMols:
    product_mol = Chem.ReplaceSubstructs(core_mol, Chem.MolFromSmarts('[#0]'), chainMol)
    product_smi.append(Chem.MolToSmiles(product_mol[0]))
print(product_smi)

In [ ]:
# View the enumerated molecules:
Draw.MolsToGridImage([Chem.MolFromSmiles(smi) for smi in product_smi])

In [ ]:
IPythonConsole.drawOptions.addAtomIndices = False

# Descriptors

## 2D Depictions

"The RDKit has a library for generating depictions (sets of 2D) coordinates for 
molecules. This library, which is part of the AllChem module, is accessed using 
the rdkit.Chem.rdDepictor.Compute2DCoords() function."

2D conformers are constructed in a canonical orientation to minimize 
intramolecular clashes, i.e. to maximize the clarity of the drawing.

See https://github.com/rdkit/rdkit/blob/master/Docs/Notebooks/FingerprintGenerators.ipynb 

In [ ]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
m = Chem.MolFromSmiles('c1nccc2n1ccc2')
AllChem.Compute2DCoords(m)

# You can save images of these molecules, or pickel them
# Draw.MolToFile(m, 'molecule.png')
# pickle.dump(m, open('moleculte.pkl', 'wb'))

In [ ]:
# For visual clarity sometimes it is helpful to show molecules with the same substructure aligned
template = Chem.MolFromSmiles('c1nccc2n1ccc2')
AllChem.Compute2DCoords(template)

ms = [Chem.MolFromSmiles(smi) for smi in ('OCCc1ccn2cnccc12','C1CC1Oc1cc2ccncn2c1','CNC(=O)c1nccc2cccn12')]

In [ ]:
# A PIL image is produced from this
img = Draw.MolsToGridImage(ms)

img
# img.save('image.png') 

In [ ]:
for m in ms:
  _ = AllChem.GenerateDepictionMatching2DStructure(m, template) # Modified in-place
Draw.MolsToGridImage(ms)

## Molecule Hash Strings

In [ ]:
from rdkit import Chem
from rdkit.Chem import rdMolHash
import rdkit

IPythonConsole.drawOptions.addAtomIndices = True

In [ ]:
s = Chem.MolFromSmiles('CC(C(C1=CC(=C(C=C1)O)O)O)N(C)C(=O)OCC2=CC=CC=C2')
s

In [ ]:
#  View all of the MolHash hashing functions types with the names method.
molhashf = rdMolHash.HashFunction.names
molhashf

In [ ]:
# Generate MolHashes for molecule 's' with all defined hash functions.
for i, j in molhashf.items():
    print(i, rdMolHash.MolHash(s, j))

## Wiener index

The [Wiener index](https://en.wikipedia.org/wiki/Wiener_index) is a topological index for a molecule, defined as the sum of the lengths of the shortest paths between all pairs of vertices in the chemical graph representing the non-hydrogen atoms in the molecule.

In [ ]:
from rdkit import Chem
def wiener_index(m):
    res = 0
    amat = Chem.GetDistanceMatrix(m)
    num_atoms = m.GetNumAtoms()
    for i in range(num_atoms):
        for j in range(i+1,num_atoms):
            res += amat[i][j]
    return res

In [ ]:
butane = Chem.MolFromSmiles('CCCC')
print(wiener_index(butane))

In [ ]:
Chem.GetDistanceMatrix(butane)

In [ ]:
butane

In [ ]:
isobutane = Chem.MolFromSmiles('CC(C)C')
print(wiener_index(isobutane))

In [ ]:
Chem.GetDistanceMatrix(isobutane)

In [ ]:
isobutane

## Tanimoto similarity